## MongoDB

MongoDB is a document database. It stores data in a type of JSON format called BSON. Here the data that is accessed together is stored together resulting in better access speed. 
A database server may have multiple databases. Once connected with a DB server we can check the DBs inside the server as follows.

In [62]:
from pymongo import MongoClient

client = MongoClient(host="localhost", port=27017)

(list(client.list_databases()))

[{'name': 'AirBNB', 'sizeOnDisk': 2539520, 'empty': False},
 {'name': 'admin', 'sizeOnDisk': 40960, 'empty': False},
 {'name': 'config', 'sizeOnDisk': 73728, 'empty': False},
 {'name': 'local', 'sizeOnDisk': 40960, 'empty': False}]

PrettyPrinter is used to format the output in a better format.

In [63]:
from pprint import PrettyPrinter
pp = PrettyPrinter(indent=2)
pp.pprint(list(client.list_databases()))

[ {'empty': False, 'name': 'AirBNB', 'sizeOnDisk': 2539520},
  {'empty': False, 'name': 'admin', 'sizeOnDisk': 40960},
  {'empty': False, 'name': 'config', 'sizeOnDisk': 73728},
  {'empty': False, 'name': 'local', 'sizeOnDisk': 40960}]


Each DB may have multiple collections similar to each SQL database having multiple Tables.


In [64]:
db = client["AirBNB"] # a database is a container for collections
db.list_collection_names()

['NewJersey']

In [65]:
jersey = db["NewJersey"]
jersey.count_documents({}) # a container has documents (records)

1726

MongoDB is a document database which is often referred to as a non-relational database.  A document in MongoDB is a record, which is a data structure composed of key value pairs similar to the structure of JSON objects. See one record below.

Field values may include numbers, strings, booleans, arrays, DateTime or even nested documents.
MongoDB stores data in flexible documents. Instead of having multiple tables you can simply keep all of your related data together. This makes reading your data very fast. 

In [66]:
# Looking at one document
result = jersey.find_one()
result

{'_id': ObjectId('66a06e06f285518eb284bc3f'),
 'id': 40669,
 'listing_url': 'https://www.airbnb.com/rooms/40669',
 'scrape_id': 20240624164330,
 'last_scraped': datetime.datetime(2024, 6, 26, 0, 0),
 'source': 'city scrape',
 'name': 'Skyy’s Lounge / Cozy',
 'neighborhood_overview': 'The neighborhood is very diverse & friendly shopping/restaurants on Central Avenue absolutely adorable very safe to walk to. The Neighborhood has a beautiful park walking distance ❤️ read a book ride your bike , rollerblade or just simply relax along the resting area ❤️ the park also has a pool / skating rink/ ice hockey very nice, this park is called Pershing Fields NJ',
 'picture_url': 'https://a0.muscache.com/pictures/af7e4a45-0118-4e9b-81c2-de396140d1ae.jpg',
 'host_id': 175412,
 'host_url': 'https://www.airbnb.com/users/show/175412',
 'host_name': 'Skyy',
 'host_since': datetime.datetime(2010, 7, 20, 0, 0),
 'host_location': 'Jersey City, NJ',
 'host_about': 'I am the owner of a high end Nail Salon in

In [67]:
# Hosts are located in how many locations
len(jersey.distinct("host_location")), jersey.distinct("host_location")[0:5]

(59,
 ['Anaheim, CA',
  'Bayonne, NJ',
  'Boston, MA',
  'Buffalo, NY',
  'California, United States'])

In [68]:
# counting docs with specific conditions, filters
jersey.count_documents({"has_availability":True})/jersey.count_documents({})

0.9947856315179606

In [69]:
# Aggregating data, counting, pivot table
result = jersey.aggregate(
    # Here's the `$` and the `"_id"`
    [
        {
            "$group": # we have  to put $ while creating anything new
            {
                "_id": "$host_location", # _id is mandatory while grouping
                "countOfHosts": {"$count": {} } 
            }
        }
    ]
)
pp.pprint(list(result)[1:10])

[ {'_id': 'Teaneck, NJ', 'countOfHosts': 1},
  {'_id': 'West New York, NJ', 'countOfHosts': 9},
  {'_id': 'United States', 'countOfHosts': 9},
  {'_id': 'North Bergen, NJ', 'countOfHosts': 4},
  {'_id': 'Los Angeles, CA', 'countOfHosts': 3},
  {'_id': 'South Lake Tahoe, CA', 'countOfHosts': 1},
  {'_id': 'Keene Valley, NY', 'countOfHosts': 4},
  {'_id': 'Chicago, IL', 'countOfHosts': 6},
  {'_id': 'San Ramon, CA', 'countOfHosts': 1}]


Comparison query operators that can be helpful for filtering the data

$gt: greater than (>)

$lt: less than (<) 

$gte: greater than equal to (>=) 

$lte: less than equal to (<= )

$eq: Values are equal

$ne: Values are not equal

$in: Value is matched within an array

In [70]:
result = jersey.find({"review_scores_rating": {"$gt": 4.7}})
pp.pprint(len(list(result)))

920


Aggregation operations allow you to group, sort, perform calculations, analyze data.

Aggregation pipelines can have one or more "stages". The order of these stages are important. Each stage acts upon the results of the previous stage.

While aggregating $match can be used to filter, $group can be used for grouping, 

In [71]:
result = jersey.aggregate(
    [    # this is like a pipeline
        {"$match": {"host_location": "Hawthorne, NJ"}}, # 38 entries here
        {"$match": {"review_scores_rating": {"$gt": 4.7}}}, #980 entries here, AND operation getting done
        {"$group": {"_id": "$review_scores_rating", "count": {"$count": {}}}} # 25 documents qualify
    ]
)

pp.pprint(list(result))

[ {'_id': 4.8, 'count': 1},
  {'_id': 4.92, 'count': 1},
  {'_id': 5.0, 'count': 17},
  {'_id': 4.81, 'count': 1},
  {'_id': 4.78, 'count': 1},
  {'_id': 4.86, 'count': 1},
  {'_id': 4.71, 'count': 2},
  {'_id': 4.75, 'count': 1}]


The following operators can logically compare multiple queries.

$and: Returns documents where both queries match

$or: Returns documents where either query matches

$nor: Returns documents where both queries fail to match

$not: Returns documents where the query does not match

The following operators assist in evaluating documents.

$regex: Allows the use of regular expressions when evaluating field values

$text: Performs a text search

$where: Uses a JavaScript expression to match documents